In [ ]:
# kagle notebook에서 데이터셋과 함께 확인 가능합니다
# link : https://www.kaggle.com/code/being0606/dh-compettition1-teamgrinder

In [30]:
!pip install torch torchvision efficientnet_pytorch onnx onnxruntime

^C
ERROR: Operation cancelled by user


데이터 로드 및 전처리

In [31]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler

# 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 데이터셋 로드
train_data = datasets.ImageFolder('../dhdata-rev/train/train', transform=transform)
val_data = datasets.ImageFolder('../dhdata-rev/validation/test', transform=transform)

# 오버샘플링을 위한 클래스 가중치 계산
class_weights = []
for root, subdir, files in os.walk('../input/dhdata-rev/train/train'):
    if len(files) > 0:
        class_weights.append(1 / len(files))

sample_weights = [class_weights[e] for e in train_data.targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# 데이터 로더 생성
train_loader = DataLoader(train_data, batch_size=16, sampler=sampler)
val_loader = DataLoader(val_data, batch_size=16)

모델구성

In [32]:
import torch
from efficientnet_pytorch import EfficientNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# EfficientNet 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b0')

# 기존 마지막 레이어의 입력 특성 수 가져오기
num_ftrs = model._fc.in_features

# 마지막 레이어에 Softmax를 포함하는 새로운 Sequential 모듈 정의
model._fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 4),  # 4개의 출력 클래스
    torch.nn.Softmax(dim=1)  # dim=1은 클래스 차원에 softmax를 적용한다는 의미 -> f1-score의 prob구하기위함.
)

# 모델을 지정된 디바이스로 이동
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


모델학습

In [33]:
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
import time


criterion = torch.nn.CrossEntropyLoss().to(device) # 필요한 경우, 손실 함수를 GPU로 이동
optimizer = optim.Adam(model.parameters())
    
    
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20, device='cuda'):
    model.to(device)
    best_val_loss = float('inf')
    best_val_f1 = 0.0
    best_model_state = None
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()

        # 학습 진행 상황 표시를 위한 tqdm
        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)

        for inputs, labels in train_progress:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # 학습 진행 상황 업데이트
            train_progress.set_postfix(loss=running_loss/len(train_loader), refresh=True)

        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

        # 검증 단계
        val_loss, val_f1 = evaluate_model(model, criterion, val_loader, device)
        print(f'Validation Loss: {val_loss:.4f}, Validation F1 Score: {val_f1:.4f}')
        
        # ETA 계산 및 출력
        time_elapsed = time.time() - start_time
        eta = time_elapsed * (num_epochs - epoch - 1)
        print(f'ETA: {eta//60:.0f}m {eta%60:.0f}s')
        
        # 검증 단계에서의 성능 개선 여부 확인
        if val_loss < best_val_loss or val_f1 > best_val_f1:
            best_val_loss = min(val_loss, best_val_loss)
            best_val_f1 = max(val_f1, best_val_f1)
            best_model_state = model.state_dict()
            print(f'New best model found! Validation Loss: {best_val_loss:.4f}, Validation F1 Score: {best_val_f1:.4f}')
        
    return model

def evaluate_model(model, criterion, data_loader, device='cuda'):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='weighted')
    return running_loss / len(data_loader), f1

In [ ]:
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20, device=device)

Epoch 1/20, Loss: 0.8825
Validation Loss: 1.0984, Validation F1 Score: 0.5211
ETA: 52m 58s
New best model found! Validation Loss: 1.0984, Validation F1 Score: 0.5211


Epoch 2/20:  34%|███▍      | 165/485 [00:50<01:37,  3.27it/s, loss=0.294]

모델 ONNX형식으로 저장

In [ ]:
model = model.to('cpu')
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "efficientnet-b0-epoch20.onnx")


추론 수행( 이후는 선택 )

In [ ]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

model.eval()  # 모델을 평가 모드로 설정

# 클래스 레이블 정의
classes = ['NoOsteoarthritis', 'NoPneumonia', 'Osteoarthritis', 'Pneumonia']
class_labels = {i: classes[i] for i in range(len(classes))}

def predict_test_img_pytorch(path):
    img = cv2.imread(path)
    print('Original Shape : ', img.shape)

    # 이미지 전처리
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0  # 정규화
    
    print('Resized Shape : ',img.shape)
    plt.imshow(img)
    plt.show()
    
    
    img = np.transpose(img, (2, 0, 1))  # PyTorch는 (C, H, W) 형식을 사용
    img = np.expand_dims(img, axis=0)  # 배치 차원 추가
    img_tensor = torch.tensor(img, dtype=torch.float32)  # NumPy 배열을 PyTorch 텐서로 변환

    # 추론 수행
    with torch.no_grad():  # 그래디언트 계산 비활성화
        outputs = model(img_tensor)
        probabilities = torch.softmax(outputs, dim=1)  # 결과를 확률로 변환
        predicted_class = torch.argmax(probabilities, dim=1)  # 가장 높은 확률을 가진 클래스 선택

    print(f'Predicted class: {class_labels[predicted_class.item()]}')
    probabilities = probabilities.numpy().flatten()  # 확률값을 1D NumPy 배열로 변환
    plt.bar(class_labels.values(), probabilities)
    plt.title('Class Probabilities')
    plt.xlabel('Class')
    plt.ylabel('Probability')
    plt.show()

In [ ]:
# 이미지 경로 지정 및 추론 함수 호출
image_path = '../dhdata-rev/test/test/0_0_02.jpg'  # 이미지 파일 경로를 지정하세요
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
predict_test_img_pytorch(image_path)